# Chapter 1: Python Data Model
---

## Special Methods

The first thing to know about special methods is that they are meant to be called by
the Python interpreter, and not by you. You don’t write `my_object.__len__()`. You
write `len(my_object)` and, if `my_object` is an instance of a user-defined class, then
Python calls the `__len__` method you implemented.

### Compare runtime of special and manual method

Python variable-sized collections
written in C include a struct2 called PyVarObject, which has an ob_size field holding
the number of items in the collection. So, if my_object is an instance of one of those
built-ins, then len(my_object) retrieves the value of the ob_size field, and this is
much faster than calling a method.

In [5]:
import timeit

# Built-in list
my_list = list(range(1000))

# Custom class with a user-defined method
class MyCollection:
    def __init__(self, items):
        self.items = items

    def size_udf(self):
        return len(self.items)

    def __len__(self):
        return len(self.items)                

my_obj = MyCollection(my_list)

# Time native len() on list
builtin_len_time = timeit.timeit('len(my_list)', globals=globals(), number=1_000_000)

# Time special method via len() on custom object
special_len_time = timeit.timeit('len(my_obj)', globals=globals(), number=1_000_000)

# Time user-defined method call
custom_size_time = timeit.timeit('my_obj.size_udf()', globals=globals(), number=1_000_000)

# print statements
print(f"Time of native len() on list        - len(my_list):       {builtin_len_time:.6f} seconds")
print(f"Time of special method via len()    - len(my_obj):        {special_len_time:.6f} seconds")
print(f"Time of user-defined method call    - my_obj.size_udf():  {custom_size_time:.6f} seconds")


Time of native len() on list        - len(my_list):       0.017417 seconds
Time of special method via len()    - len(my_obj):        0.056149 seconds
Time of user-defined method call    - my_obj.size_udf():  0.029865 seconds


Normally, your code should not have many direct calls to special methods. Unless
you are doing a lot of metaprogramming

### Emulating Numeric Types

![Figure 1](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/3.PNG)


#### Example 1-2. A simple two-dimensional vector class

In [ ]:
"""
vector2d.py: a simplistic class demonstrating some special methods

It is simplistic for didactic reasons. It lacks proper error handling,
especially in the ``__add__`` and ``__mul__`` methods.

Addition::
    >>> v1 = Vector(2, 4)
    >>> v2 = Vector(2, 1)
    >>> v1 + v2
    Vector(4, 5)

Absolute value::

    >>> v = Vector(3, 4)
    >>> abs(v)
    5.0

Scalar multiplication::

    >>> v * 3
    Vector(9, 12)
    >>> abs(v * 3)
    15.0
"""

import math

class Vector:

    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        #return 'Vector(%r, %r)' % (self.x, self.y)
        return f"Vector({self.x!r}, {self.y!r})"

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)

    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

In [18]:
v1 = Vector(2, 4)
v2 = Vector(2, 1)
v1 + v2

Vector(4, 5)

In [19]:
v = Vector(3, 4)
abs(v)

5.0

In [20]:
v * 3

Vector(9, 12)

In [21]:
abs(v * 3)

15.0

**Note**: On function think of x!r as f"{x!r}"  ==  f"{repr(x)}"

- `!r` = use repr(x)

- `!s` = use str(x) (default, so usually you can omit it)

- `!a` = use ascii(x) (for escaped representations)

```python
x = 'hello\nworld'

print(str(x))   # hello
                # world

print(repr(x))  # 'hello\nworld'
```